<a href="https://colab.research.google.com/github/jiobu1/DS-Unit-2-Linear-Models/blob/master/Jisha_Obukwelu_DSPT3_212_Regression_Classification_2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to our fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [x] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [x] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [x] Add your own stretch goal(s) !
- created function to calculate all in 1 step. 
- try to see visually the boroughs (saw this on a previous video)

####Taking care of imports

In [0]:
import numpy as np
import pandas as pd

import itertools

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

* Data Exploration
- what data can be combined and 

In [0]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 1. Engineer at least two new features. 




* Already engineered one feature in my previous notebook, Haversine point (takes longitude and latitude and maps as a single point taking into account Earth's curvature)
* Total number of rooms (beds + baths)
* Luxury rentals - How many total perks does each apartment have?
* Are cats or dogs allowed?
* Ratio of beds to baths
* What's the neighborhood, based on address or latitude & longitude?




In [0]:
#Haversine point
from math import radians, cos, sin, asin, sqrt

def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d

In [0]:
df['haversine'] = [single_pt_haversine(x, y) for x, y in zip(df.latitude, df.longitude)]

##### Adding the number of bathrooms and bedrooms together





In [0]:
#Number of rooms
df['rooms'] = df['bathrooms'] + df['bedrooms']

##### Changing interest level to numeric using .replace()

In [0]:
df['interest_level'].unique()

array(['medium', 'low', 'high'], dtype=object)

In [0]:
df['interest_level'] = df['interest_level'].replace(['low', 'medium', 'high'], [1, 2, 3])

In [0]:
df['interest_level'].unique()

array([2, 1, 3])

#####Are pets allowed \
(I am going to add combine the 2 columns - 1, 0 for whether or not animals are allowed)

In [0]:
df['cats_allowed'].sum()

23348

In [0]:
df['dogs_allowed'].sum()

21852

In [0]:
def pets_allowed(x):
  if x['cats_allowed'] == 1:
    return 1
  elif x['dogs_allowed'] == 1:
    return 1
  else:
    return 0
  
df['pets_allowed'] = df.apply(pets_allowed, axis=1)
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,haversine,rooms,pets_allowed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8661.842498,4.5,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8665.491494,3.0,1
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8667.251778,2.0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8664.737441,2.0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8664.656031,5.0,0


In [0]:
#dropping cats_allowed/dogs_allowed columns
df = df.drop(['cats_allowed', 'dogs_allowed'], axis=1)

In [0]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,hardwood_floors,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,haversine,rooms,pets_allowed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8661.842498,4.5,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8665.491494,3.0,1
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,3,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8667.251778,2.0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8664.737441,2.0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8664.656031,5.0,0


##### Creating a luxury rating scale - adding all the features available to get a scale for ammenities

In [0]:
#looking at what features I want to include in the luxury rating
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'hardwood_floors', 'doorman', 'dishwasher', 'no_fee',
       'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit',
       'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet',
       'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive',
       'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space',
       'haversine', 'rooms', 'pets_allowed'],
      dtype='object')

In [0]:
#going to reorder the columns to make addition easier. 
#also, I am unsure what exclusive mean - you're the only one with access to your apartment?
#dropping display address - it seems street address is more informative
df = df[['created', 'street_address', 'description', 
         'latitude', 'longitude', 'haversine',
         'price', 'interest_level', 
         'no_fee', 'exclusive', 
         'bathrooms', 'bedrooms',
         'pre-war', 'new_construction','loft',
         'pets_allowed', 'hardwood_floors','dining_room','laundry_in_unit', 
         'dishwasher', 'high_speed_internet', 'balcony','terrace',
         'elevator', 'doorman', 'laundry_in_building',
         'fitness_center', 'swimming_pool',
         'roof_deck', 'outdoor_space', 'garden_patio',
         'common_outdoor_space', 'wheelchair_access','rooms']]

In [0]:
#checking it is in the right order
df.head()

,created,street_address,description,latitude,longitude,haversine,price,interest_level,no_fee,exclusive,bathrooms,bedrooms,pre-war,new_construction,loft,pets_allowed,hardwood_floors,dining_room,laundry_in_unit,dishwasher,high_speed_internet,balcony,terrace,elevator,doorman,laundry_in_building,fitness_center,swimming_pool,roof_deck,outdoor_space,garden_patio,common_outdoor_space,wheelchair_access,rooms
0,2016-06-24 07:54:24,792 Metropolitan Avenue,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,40.7145,-73.9425,8661.842498,3000,2,0,0,1.5,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5
1,2016-06-12 12:19:27,808 Columbus Avenue,,40.7947,-73.9667,8665.491494,5465,1,0,0,1.0,2,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,3.0
2,2016-04-17 03:26:41,241 W 13 Street,"Top Top West Village location, beautiful Pre-w...",40.7388,-74.0018,8667.251778,2850,3,0,0,1.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,2.0
3,2016-04-18 02:22:02,333 East 49th Street,Building Amenities - Garage - Garden - fitness...,40.7539,-73.9677,8664.737441,3275,1,1,0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0
4,2016-04-28 01:32:41,500 West 143rd Street,Beautifully renovated 3 bedroom flex 4 bedroom...,40.8241,-73.9493,8664.656031,3350,1,0,0,1.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0


In [0]:
df.shape

(48817, 34)

In [0]:
#Luxury score - this includes interest level and bathroom/bedroom
df['luxury_rating'] = sum(df.iloc[:,7:33].values.T)

In [0]:
df.luxury_rating.head()

0    6.5
1    8.0
2    8.0
3    5.0
4    7.0
Name: luxury_rating, dtype: float64

In [0]:
#what are the different scores
min(df['luxury_rating']), max(df['luxury_rating'])

(1.0, 26.0)

##### Checking if there are null values

In [0]:
df.isnull().sum()

created                    0
street_address            10
description             1425
latitude                   0
longitude                  0
haversine                  0
price                      0
interest_level             0
no_fee                     0
exclusive                  0
bathrooms                  0
bedrooms                   0
pre-war                    0
new_construction           0
loft                       0
pets_allowed               0
hardwood_floors            0
dining_room                0
laundry_in_unit            0
dishwasher                 0
high_speed_internet        0
balcony                    0
terrace                    0
elevator                   0
doorman                    0
laundry_in_building        0
fitness_center             0
swimming_pool              0
roof_deck                  0
outdoor_space              0
garden_patio               0
common_outdoor_space       0
wheelchair_access          0
rooms                      0
luxury_rating 

#### Checking the correlation matrix to see what features most affect price.

In [0]:
corr_matrix=df.corr()
corr_matrix['price'].sort_values(ascending=False)

price                   1.000000
bathrooms               0.687296
rooms                   0.649097
bedrooms                0.535503
luxury_rating           0.464329
doorman                 0.276215
laundry_in_unit         0.271195
haversine               0.252851
dining_room             0.242911
fitness_center          0.228775
dishwasher              0.223899
elevator                0.207169
terrace                 0.145973
outdoor_space           0.142146
balcony                 0.139140
swimming_pool           0.134513
no_fee                  0.132240
roof_deck               0.122929
garden_patio            0.103672
hardwood_floors         0.101503
high_speed_internet     0.090269
wheelchair_access       0.072517
new_construction        0.071431
pets_allowed            0.050989
common_outdoor_space    0.011517
loft                    0.007100
exclusive              -0.013251
laundry_in_building    -0.019417
pre-war                -0.029122
latitude               -0.036286
interest_l

#### 2. Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.





*   Changing created by column into date time format
*   Creating a months column to separate easier



In [141]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created'].head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: created, dtype: datetime64[ns]

In [0]:
train = df[df['created'] <'06-01-2016']
test = df[df['created'] >='06-01-2016']



*   Checking that the test and train dataframes have the appropriate months



In [143]:
min(train['created']), max(train['created']), min(test['created']), max(test['created'])

(Timestamp('2016-04-01 22:12:41'),
 Timestamp('2016-05-31 23:10:48'),
 Timestamp('2016-06-01 01:10:37'),
 Timestamp('2016-06-29 21:41:47'))

* Checking train/test shape

In [144]:
print(train.shape, test.shape)

(31844, 35) (16973, 35)


In [145]:
train.describe()

,latitude,longitude,haversine,price,interest_level,no_fee,exclusive,bathrooms,bedrooms,pre-war,new_construction,loft,pets_allowed,hardwood_floors,dining_room,laundry_in_unit,dishwasher,high_speed_internet,balcony,terrace,elevator,doorman,laundry_in_building,fitness_center,swimming_pool,roof_deck,outdoor_space,garden_patio,common_outdoor_space,wheelchair_access,rooms,luxury_rating
count,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000
mean,40.750743,-73.972867,8665.101359,3575.604007,1.383149,0.369834,0.042269,1.203728,1.528357,0.185938,0.051470,0.044216,0.477861,0.480907,0.104290,0.175700,0.418666,0.088620,0.060734,0.047733,0.530430,0.430725,0.057311,0.267586,0.055929,0.133777,0.143983,0.039222,0.029048,0.028388,2.732085,8.379867
std,0.038658,0.028910,2.260512,1762.136694,0.622672,0.482767,0.201204,0.472447,1.105061,0.389062,0.220957,0.205577,0.499517,0.499643,0.305641,0.380571,0.493348,0.284198,0.238845,0.213203,0.499081,0.495185,0.232439,0.442707,0.229788,0.340418,0.351078,0.194127,0.167943,0.166082,1.411986,3.895025
min,40.575700,-74.087300,8640.617270,1375.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,40.728500,-73.991800,8664.098930,2500.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,5.000000
50%,40.751700,-73.978100,8665.466457,3150.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,7.500000
75%,40.773600,-73.955000,8666.628142,4095.000000,2.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,11.000000
max,40.910200,-73.700100,8673.082490,15500.000000,3.000000,1.000000,1.000000,10.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,26.000000


In [146]:
test.describe()

,latitude,longitude,haversine,price,interest_level,no_fee,exclusive,bathrooms,bedrooms,pre-war,new_construction,loft,pets_allowed,hardwood_floors,dining_room,laundry_in_unit,dishwasher,high_speed_internet,balcony,terrace,elevator,doorman,laundry_in_building,fitness_center,swimming_pool,roof_deck,outdoor_space,garden_patio,common_outdoor_space,wheelchair_access,rooms,luxury_rating
count,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.000000,16973.00000,16973.000000,16973.000000
mean,40.750791,-73.972560,8665.076928,3587.054675,1.374713,0.361928,0.045248,1.198168,1.553644,0.185118,0.052731,0.039887,0.481176,0.473340,0.100100,0.176280,0.408354,0.084546,0.059978,0.043304,0.514346,0.413834,0.044247,0.270076,0.053850,0.130855,0.127909,0.039533,0.020974,0.02504,2.751812,8.279179
std,0.039503,0.028831,2.254308,1763.010038,0.620607,0.480572,0.207854,0.467429,1.107854,0.388404,0.223502,0.195699,0.499660,0.499303,0.300142,0.381069,0.491544,0.278213,0.237452,0.203547,0.499809,0.492534,0.205649,0.444012,0.225729,0.337252,0.333998,0.194866,0.143303,0.15625,1.408897,3.849490
min,40.575700,-74.045400,8643.660743,1375.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000
25%,40.727700,-73.991700,8664.083632,2500.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.000000,5.000000
50%,40.751600,-73.977700,8665.461476,3150.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.500000,7.000000
75%,40.774400,-73.955000,8666.590698,4095.000000,2.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,4.000000,10.500000
max,40.989400,-73.714200,8670.666894,15500.000000,3.000000,1.000000,1.000000,5.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,12.000000,26.000000


####3a. Fit a linear regression model with at least two features.
#### 3b. Get the model's coefficients and intercept.
#### 4. Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.



* Baseline of 0 features

In [147]:
#Arrange y target vectors
target = 'price'
y_train = train[target]
y_test = test[target]

#Get mean baseline
print('Mean baseline (using 0 features)')
guess = y_train.mean()

#Train Error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (NYC - April and May rents): ${mae:.2f}')

#Test Error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error (NYC - June rent): ${mae:.2f}')

Mean baseline (using 0 features)
Train Error (NYC - April and May rents): $1201.88
Test Error (NYC - June rent): $1197.71


This is our error with no features chosen. Just if we guess the mean price.

##### Function to calculate 
* fit linear regression with at least two features
* model coefficient, 
* intercept, 
* MAE, 
* RMSE, 
* R2

In [149]:
df.columns

Index(['created', 'street_address', 'description', 'latitude', 'longitude',
       'haversine', 'price', 'interest_level', 'no_fee', 'exclusive',
       'bathrooms', 'bedrooms', 'pre-war', 'new_construction', 'loft',
       'pets_allowed', 'hardwood_floors', 'dining_room', 'laundry_in_unit',
       'dishwasher', 'high_speed_internet', 'balcony', 'terrace', 'elevator',
       'doorman', 'laundry_in_building', 'fitness_center', 'swimming_pool',
       'roof_deck', 'outdoor_space', 'garden_patio', 'common_outdoor_space',
       'wheelchair_access', 'rooms', 'luxury_rating'],
      dtype='object')

#####Trying all features

In [0]:
features = ['latitude', 'longitude','haversine', 'interest_level', 'no_fee', 
            'exclusive','bathrooms', 'bedrooms', 'pre-war', 'new_construction', 
            'loft', 'pets_allowed', 'hardwood_floors', 'dining_room', 
            'laundry_in_unit','dishwasher', 'high_speed_internet', 'balcony', 
            'terrace', 'elevator', 'doorman', 'laundry_in_building', 
            'fitness_center', 'swimming_pool','roof_deck', 'outdoor_space', 
            'garden_patio', 'common_outdoor_space','wheelchair_access', 'rooms', 
            'luxury_rating']
target = 'price'

In [0]:
def train_test():
  # Instantiate the class
  model = LinearRegression()

  # Arrange x features matrices and y target vectors
  X_train = train[features]
  X_test = test[features]
  y_train = train[target]
  y_test = test[target]
  print(f'Linear Regression dependent on:\n {len(features)}\n {features}, \n')

  #Scale the data
  X_train_scaled = StandardScaler().fit_transform(X_train)
  X_test_scaled = StandardScaler().fit_transform(X_test)

  #Fit the model
  model.fit(X_train_scaled, y_train)
  y_pred_train = model.predict(X_train_scaled)
  # model.fit(X_train, y_train)
  # y_pred_train = model.predict(X_train)
  train_error = mean_absolute_error(y_train, y_pred_train)
  
  #Apply the model to new data
  y_pred_test = model.predict(X_test_scaled)
  # y_pred_test = model.predict(X_test)
  test_error = mean_absolute_error(y_test, y_pred_test)  

  
  #Regression metrics
  # Mean Squared Error
  # Root Mean Squared Error
  # R^2

  #Train
  train_mse = mean_squared_error(y_train, y_pred_train)
  train_rmse = np.sqrt(train_mse)
  train_r2 = r2_score(y_train, y_pred_train)

  #Test
  test_mse = mean_squared_error(y_test, y_pred_test)
  test_rmse = np.sqrt(test_mse)
  test_r2 = r2_score(y_test, y_pred_test)

  #Taking care of all my prints
  
  #Training Data:
  print('Train Data:')
  print(f'Train MAE: ${train_error:.2f}')
  print(f'Train MSE: ${train_mse:.2f}')
  print(f'Train RMSE: ${train_rmse:.2f}')
  print(f'Train R^2: {train_r2:.2f} \n')

  #Testing Data
  print('Test Data:')
  print(f'Test MAE ${test_error:.2f}')
  print(f'Test MSE: ${test_mse:.2f}')
  print(f'Test RMSE: ${test_rmse:.2f}')
  print(f'Test R^2: {test_r2:.2f} \n')

  #Model Interecept and Coefficients
  print('Model Intercept and Coefficients \n')
  print(f'Model intercept {model.intercept_} \n')
  print(f'Coefficients \n{pd.Series(model.coef_, features)}')

When I scale the features, I get an R^2 of -29 but when I do not I have a R^2 of 65.

In [152]:
train_test()

Linear Regression dependent on:
 31
 ['latitude', 'longitude', 'haversine', 'interest_level', 'no_fee', 'exclusive', 'bathrooms', 'bedrooms', 'pre-war', 'new_construction', 'loft', 'pets_allowed', 'hardwood_floors', 'dining_room', 'laundry_in_unit', 'dishwasher', 'high_speed_internet', 'balcony', 'terrace', 'elevator', 'doorman', 'laundry_in_building', 'fitness_center', 'swimming_pool', 'roof_deck', 'outdoor_space', 'garden_patio', 'common_outdoor_space', 'wheelchair_access', 'rooms', 'luxury_rating'], 

Train Data:
Train MAE: $659.39
Train MSE: $1098861.31
Train RMSE: $1048.27
Train R^2: 0.65 

Test Data:
Test MAE $1466.16
Test MSE: $4023750.74
Test RMSE: $2005.93
Test R^2: -0.29 

Model Intercept and Coefficients 

Model intercept 3575.6040069770625 

Coefficients 
latitude                 69380.239800
longitude              -209682.171909
haversine              -197629.969929
interest_level            -257.543405
no_fee                     -69.075088
exclusive                   26.7

If some coefficients are negatively aligned, should I exclude it in my training/testing data?

In [0]:
#Changing the features based on the correlation matrix
features = ['bathrooms', 'rooms', 'bedrooms', 'luxury_rating', 'doorman',
            'laundry_in_unit', 'haversine', 'dining_room', 'fitness_center', 
            'dishwasher', 'elevator', 'dishwasher', 
            'elevator', 'terrace', 'outdoor_space', 'balcony', 
            'swimming_pool', 'no_fee', 'roof_deck', 'garden_patio', 'hardwood_floors',
            'high_speed_internet', 'wheelchair_access', 'new_construction', 'pets_allowed', 
            'common_outdoor_space', 'loft']

In [156]:
train_test()

Linear Regression dependent on:
 27
 ['bathrooms', 'rooms', 'bedrooms', 'luxury_rating', 'doorman', 'laundry_in_unit', 'haversine', 'dining_room', 'fitness_center', 'dishwasher', 'elevator', 'dishwasher', 'elevator', 'terrace', 'outdoor_space', 'balcony', 'swimming_pool', 'no_fee', 'roof_deck', 'garden_patio', 'hardwood_floors', 'high_speed_internet', 'wheelchair_access', 'new_construction', 'pets_allowed', 'common_outdoor_space', 'loft'], 

Train Data:
Train MAE: $682.87
Train MSE: $1152104.63
Train RMSE: $1073.36
Train R^2: 0.63 

Test Data:
Test MAE $3574867879725.53
Test MSE: $20076132812140575878807552.00
Test RMSE: $4480639777101.10
Test R^2: -6459457990135267328.00 

Model Intercept and Coefficients 

Model intercept 3575.541864662486 

Coefficients 
bathrooms               4.565133e+14
rooms                  -1.364364e+15
bedrooms                1.067791e+15
luxury_rating          -1.094814e+03
doorman                 3.477370e+02
laundry_in_unit         2.971560e+02
haversine 

What the heck is this number
R^2 -6459457990135267328.00  for testing, wow!

In [0]:
features = ['haversine', 'bathrooms', 'laundry_in_unit', 'doorman', 'rooms']

In [179]:
train_test()

Linear Regression dependent on:
 5
 ['haversine', 'bathrooms', 'laundry_in_unit', 'doorman', 'rooms'], 

Train Data:
Train MAE: $710.40
Train MSE: $1237484.40
Train RMSE: $1112.42
Train R^2: 0.60 

Test Data:
Test MAE $720.71
Test MSE: $1214457.77
Test RMSE: $1102.02
Test R^2: 0.61 

Model Intercept and Coefficients 

Model intercept 3575.604007034354 

Coefficients 
haversine          363.494855
bathrooms          648.646396
laundry_in_unit    152.530267
doorman            247.138607
rooms              639.476030
dtype: float64


This is more like it, not the best but definitely better.